Notebook disponível de forma interativa no [Binder](https://mybinder.org/v2/gh/QApedia/QApedia/master?filepath=examples).

# Realizando consultas sparql com o QApedia

A partir de um SPARQL Endpoint e uma consulta SPARQL válida, é possível obter o resultado da consulta utilizando o QApedia através da função ``perform_query``. 

Uma consulta SPARQL geralmente possui as seguintes partes (algumas são opcionais como a definição do dataset e os modificadores da consulta):


* **prefix declarations**
nomeia uma URI de forma a abreviar
```
PREFIX foo: <http://example.com/resources/>
...
```
* **dataset definition**
Indica os grafos RDFS consultados
```
FROM ...
```
* **result clause**
Indica a informação a ser retornada da consulta
```
SELECT ...
```
* **query pattern**
Especifica o que consultar no conjunto de dados
```
WHERE {
    ...
}
```
* **query modifiers**
 Pode reorganizar os dados da consulta, limitar a quantidade de resultados, reordenar, etc.
```
ORDER BY ...
```

No final, a consulta pode possuir o seguinte aspecto.
```
# prefix declarations
PREFIX foo: <http://example.com/resources/>
...
# dataset definition
FROM ...
# result clause
SELECT ...
# query pattern
WHERE {
    ...
}
# query modifiers
ORDER BY ...
```


A função ``perform_query`` recebe os seguintes argumentos:

* query - Corresponde a consulta SPARQL utilizada, essa consulta deve ser fornecida no formato de uma string.
* prefixes - Conjunto de prefixos utilizados  na consulta SPARQL. 
* endpoint - SPARQL Endpoint onde a busca será realizar.

A seguir, é mostrado alguns exemplos de como realizar consultas com o QApedia. 

In [1]:
# carregar módulo generator
from QApedia import generator

a) Obter todas as pessoas do arquivo FOAF de Tim Berners-Lee que tenham nomes e endereço de email. Para cada resultado é retornado o URI, o nome e o email de cada pessoa. A consulta pode ser realizada utilizando o [OpenLink's Virtuoso](http://demo.openlinksw.com/sparql).

```
PREFIX foaf:  <http://xmlns.com/foaf/0.1/>
SELECT *
WHERE {
    ?person foaf:name ?name .
    ?person foaf:mbox ?email .
}
```

Para realizar a seguinte busca, você pode passar a consulta acima completa ou dividir em duas partes sendo elas ``prefixos`` e ``sparql``. 

Por que dividir em duas partes?
Caso esteja trabalhando com diversas consultas SPARQLs, a fim de manter o dataset padronizado, pode-se definir um conjunto de prefixos que serão reutilizados nas consultas, evitando o problema do uso de duas denominações diferentes para um mesmo URI. Em http://dbpedia.org/sparql o resource é definido por padrão como ``PREFIX dbr:<http://dbpedia.org/resource/>``, já em http://dbpedia.org/snorql é definido como ``PREFIX :<http://dbpedia.org/resource/>``.

In [2]:
prefixos = "PREFIX foaf:  <http://xmlns.com/foaf/0.1/>"
sparql = """
SELECT *
WHERE {
    ?person foaf:name ?name .
    ?person foaf:mbox ?email .
}
"""
result = generator.perform_query(sparql, prefixos, endpoint = "http://demo.openlinksw.com/sparql")
len(result)

346

**Lista dos dez primeiros retornados pela consulta**

In [3]:
result[0:10]

[{'person': Value(uri:'http://www.ils.unc.edu/~janeg/foaf.rdf#me'),
  'name': Value(literal:'Jane Greenberg'),
  'email': Value(uri:'mailto:janeg@email.unc.edu')},
 {'person': Value(uri:'http://www.ils.unc.edu/~janeg/foaf.rdf#me'),
  'name': Value(literal:'Jane Greenberg'),
  'email': Value(uri:'mailto:janeg@email.unc.edu')},
 {'person': Value(uri:'http://www.ils.unc.edu/~janeg/foaf.rdf#me'),
  'name': Value(literal:'Jane Greenberg'),
  'email': Value(uri:'mailto:janeg@email.unc.edu')},
 {'person': Value(uri:'http://www.ils.unc.edu/~janeg/foaf.rdf#me'),
  'name': Value(literal:'Jane Greenberg'),
  'email': Value(uri:'mailto:janeg@email.unc.edu')},
 {'person': Value(uri:'http://www.ils.unc.edu/~janeg/foaf.rdf#me'),
  'name': Value(literal:'Jane Greenberg'),
  'email': Value(uri:'mailto:janeg@email.unc.edu')},
 {'person': Value(uri:'http://www.ils.unc.edu/~janeg/foaf.rdf#me'),
  'name': Value(literal:'Jane Greenberg'),
  'email': Value(uri:'mailto:janeg@email.unc.edu')},
 {'person': Valu

b) Usando o endpoint da DBpedia, podemos fazer uma consulta que obtém todos os mangás escritos pelo autor Yoshihiro Togashi. A consulta pode ser realizada utilizando o [DBPedia-specific SPARQL endpoint](http://dbpedia.org/sparql).

```
SELECT * WHERE {
  ?manga a dbo:Manga .
  ?manga rdfs:label ?nome .
  ?manga dbo:author dbr:Yoshihiro_Togashi .
  FILTER(lang(?nome) = 'pt').}

```
Nessa consulta utilizamos o FILTER para retornar apenas os resultados em português.

In [4]:
sparql = """
SELECT * WHERE {
  ?manga a dbo:Manga .
  ?manga rdfs:label ?nome .
  ?manga dbo:author dbr:Yoshihiro_Togashi .
  FILTER(lang(?nome) = 'pt').}
"""
result = generator.perform_query(sparql, endpoint = "http://dbpedia.org/sparql")
result

[{'manga': Value(uri:'http://dbpedia.org/resource/Level_E'),
  'nome': Value(literal:'Level E')},
 {'manga': Value(uri:'http://dbpedia.org/resource/Yu_Yu_Hakusho'),
  'nome': Value(literal:'Yu Yu Hakusho')},
 {'manga': Value(uri:'http://dbpedia.org/resource/Hunter_×_Hunter'),
  'nome': Value(literal:'Hunter × Hunter')}]

É possível utilizar outras cláusulas de consulta SPARQL como o ASK. A biblioteca retorna verdadeiro ou falso dependendo do resultado da consulta SPARQL.

c) Usar consulta SPARQL para verificar se Yoshihiro Togashi é autor de Yu Yu Hakusho. Essa consulta deve retornar verdadeiro, visto que na consulta anterior é um dos títulos retornados.

In [2]:
sparql = """
ASK WHERE {
  dbr:Yu_Yu_Hakusho dbo:author dbr:Yoshihiro_Togashi .
}
"""
result = generator.perform_query(sparql, endpoint = "http://dbpedia.org/sparql")
result

True

c) Usar consulta SPARQL para verificar se Yoshihiro Togashi é autor de Naruto.

In [3]:
sparql = """
ASK WHERE {
  dbr:Yu_Yu_Hakusho dbo:author dbr:Naruto .
}
"""
result = generator.perform_query(sparql, endpoint = "http://dbpedia.org/sparql")
result

False